# Importamos las librerias necesarias

In [21]:
import pandas as pd
import numpy as np
import requests as rq
import json
import boto3

# Carga de archivos y conexion a API REST

## 1. Creacion de variables con data constante 

In [22]:
#Para el consumo del API REST
url_api = "https://api.themoviedb.org/3"
search_method = "/search/movie?"
api_key = "api_key=053782026411bb6b27981d421239e12c"
query = "&query="  

In [23]:
# Para los archivos CSV existentes el S3
director_file = 's3://movie.data.mubi/directors.csv'
users_file = 's3://movie.data.mubi/users.csv'
movies_file = 's3://movie.data.mubi/movies.csv'
ratings_file = 's3://movie.data.mubi/ratings.csv'
movies_onstreaming = 's3://movie.data.mubi/MoviesOnStreamingPlatforms_updated.csv'

In [4]:
response = rq.get(url_api+search_method+api_key+query+"batman")
print(response)

<Response [200]>


## 2. Carga de archivos en dataframes

In [17]:
df_directors = pd.read_csv(director_file)
df_directors.head()

,director_id,director_name
0,100,Werner Herzog
1,"100, 19053","Werner Herzog, Andre Singer"
2,"100, 602, 11891, 783, 832","Werner Herzog, Andrzej Wajda, Luigi Comencini,..."
3,"100, 81649","Werner Herzog, Denis Reichle"
4,"100006, 57351","Shelly Dunn Fremont, Vincent Fremont"


In [18]:
df_users = pd.read_csv(users_file)
df_users.head()

,user_id,user_trialist,user_subscriber
0,57,0.0,0.0
1,176,0.0,0.0
2,181,0.0,0.0
3,499,0.0,0.0
4,606,0.0,0.0


In [24]:
df_movies_onstreaming = pd.read_csv(movies_onstreaming, index_col=0)
df_movies_onstreaming = df_movies_onstreaming.iloc[:, 0:10]

In [25]:
df_movies_onstreaming.columns = ['id', 'movie_title', 'year', 'age','imdb','rotten_tomatoes','netflix', 'hulu', 'prime_video', 'disney']
df_movies_onstreaming.head()

,id,movie_title,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney
0,1,Inception,2010,13+,8.8,87%,1,0,0,0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0


In [26]:
df_movies = pd.read_csv(movies_file)
df_movies.head()

,movie_id,movie_title,movie_release_year,movie_title_language,movie_popularity,director_id
0,1,La Antena,2007.0,en,105.0,131
1,2,Elementary Particles,2006.0,en,23.0,73
2,3,It's Winter,2006.0,en,21.0,82
3,4,Kirikou and the Wild Beasts,2005.0,en,46.0,"89, 90"
4,5,Padre Nuestro,2007.0,en,7.0,92


In [10]:
#Header para los mini-dataset del archivo mas grande (+15.000.000 de registros)
header_name_rating=["rating_id","rating_score","rating_timestamp_utc","critic","critic_comments","critic_likes","user_id","movie_id"]

In [11]:
df_ratings_1 = pd.read_csv(ratings_file, nrows = 4000000)
df_ratings_1.head()

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
0,2,4.0,2016-11-06 03:34:45,NaN,0,0,49985304,35
1,3,4.0,2016-11-06 03:36:48,NaN,0,0,49985304,15
2,4,3.0,2008-02-01 01:01:43,NaN,0,0,49985304,24
3,5,5.0,2008-02-01 01:01:45,NaN,0,0,49985304,12
4,7,3.0,2016-11-06 03:37:13,NaN,0,0,49985304,3


In [11]:
df_ratings_1.iloc[-2:]

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
3999998,4588502,4.0,2011-01-24 06:49:25,NaN,0,0,96225796,920
3999999,4588503,2.0,2011-01-24 06:49:27,NaN,0,0,96225796,318


In [12]:
df_ratings_2 = pd.read_csv(ratings_file, nrows = 4000000, skiprows = 4000001,  names = header_name_rating)
df_ratings_2.head()

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
0,4588504,4.0,2011-01-24 06:49:32,NaN,0,0,96225796,2443
1,4588505,4.0,2011-01-24 06:49:35,NaN,0,0,96225796,202
2,4588506,2.0,2011-01-24 06:49:38,NaN,0,0,69978410,29251
3,4588507,3.0,2011-01-24 06:49:41,NaN,0,0,96225796,2373
4,4588508,5.0,2011-01-24 06:49:43,NaN,0,0,96225796,977


In [13]:
df_ratings_2.iloc[-2:]

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
3999998,9329980,4.0,2013-05-16 09:52:59,NaN,0,0,60272709,46284
3999999,9329981,4.0,2013-05-16 09:56:07,NaN,0,0,37361745,8383


In [13]:
df_ratings_3 = pd.read_csv(ratings_file, nrows = 4000000, skiprows = 8000001,  names = header_name_rating)
df_ratings_3.head()

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
0,9329983,5.0,2013-05-16 09:59:34,NaN,0,0,74502356,24928
1,9329984,2.0,2013-05-16 10:00:02,NaN,0,0,43913749,611
2,9329985,4.0,2013-05-16 10:00:12,NaN,0,0,37361745,3742
3,9329986,5.0,2013-05-16 10:01:49,NaN,0,0,43913749,635
4,9329987,3.0,2013-05-16 10:01:52,NaN,0,1,29828133,23406


In [15]:
df_ratings_3.iloc[-2:]

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
3999998,15337891,4.0,2017-03-26 22:32:27,NaN,0,0,46677790,149803
3999999,15337893,5.0,2017-03-26 22:35:04,NaN,0,0,59420344,151


In [14]:
df_ratings_4 = pd.read_csv(ratings_file, skiprows = 12000002,  names = header_name_rating)
df_ratings_4.head()

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
0,15337895,2.0,2017-03-26 22:33:28,NaN,0,0,30969152,19659
1,15337897,2.0,2017-03-26 22:34:00,"Tiene momentos disparatados que me recordaron,...",0,0,42848211,165214
2,15337898,4.0,2017-03-26 22:34:02,NaN,0,0,51183480,154661
3,15337899,3.0,2017-03-26 22:34:10,NaN,0,0,59586579,5158
4,15337906,5.0,2017-03-26 22:34:52,NaN,0,0,76490684,43481


In [17]:
df_ratings_4.iloc[-2:]

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
3519960,19971104,3.0,2020-04-26 09:49:57,Although not particularly innovative in terms ...,0,0,74593823,263652
3519961,19971439,3.0,2020-04-26 10:34:08,2020 A0405,0,0,28234610,262827


## 3. Descripcion de los dataset

In [18]:
df_directors.describe(include='all')

,director_id,director_name
count,95899,95897
unique,95899,95645
top,539020,Ingo Petzke
freq,1,9


In [19]:
df_users.describe(include='all')

,user_id,user_trialist,user_subscriber
count,4.517570e+05,451754.000000,451754.000000
mean,4.997782e+07,0.080962,0.230736
std,2.885431e+07,0.272777,0.421304
min,5.700000e+01,0.000000,0.000000
25%,2.503906e+07,0.000000,0.000000
50%,4.989117e+07,0.000000,0.000000
75%,7.498138e+07,0.000000,0.000000
max,9.999986e+07,1.000000,1.000000


In [20]:
df_movies_onstreaming.describe(include='all')

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
count,16744.000000,16744.000000,16744,16744.000000,7354,16173.000000,5158,16744.000000,16744.000000,16744.000000,16744.000000,16744.0,16018,16469,16309,16145,16152.000000
unique,NaN,NaN,16744,NaN,5,NaN,99,NaN,NaN,NaN,NaN,NaN,11338,1909,1303,1102,NaN
top,NaN,NaN,Jesse James Meets Frankenstein's Daughter,NaN,18+,NaN,100%,NaN,NaN,NaN,NaN,NaN,Jay Chapman,Drama,United States,English,NaN
freq,NaN,NaN,1,NaN,3474,NaN,407,NaN,NaN,NaN,NaN,NaN,36,1341,8776,10955,NaN
mean,8371.500000,8372.500000,NaN,2003.014035,NaN,5.902751,NaN,0.212613,0.053930,0.737817,0.033684,0.0,NaN,NaN,NaN,NaN,93.413447
std,4833.720789,4833.720789,NaN,20.674321,NaN,1.347867,NaN,0.409169,0.225886,0.439835,0.180419,0.0,NaN,NaN,NaN,NaN,28.219222
min,0.000000,1.000000,NaN,1902.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,1.000000
25%,4185.750000,4186.750000,NaN,2000.000000,NaN,5.100000,NaN,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,82.000000
50%,8371.500000,8372.500000,NaN,2012.000000,NaN,6.100000,NaN,0.000000,0.000000,1.000000,0.000000,0.0,NaN,NaN,NaN,NaN,92.000000
75%,12557.250000,12558.250000,NaN,2016.000000,NaN,6.900000,NaN,0.000000,0.000000,1.000000,0.000000,0.0,NaN,NaN,NaN,NaN,104.000000


In [21]:
df_movies.describe(include='all')

,movie_id,movie_title,movie_release_year,movie_title_language,movie_popularity,director_id
count,226575.000000,226575,226570.000000,226575,226575.000000,226575
unique,NaN,196628,NaN,1,NaN,95899
top,NaN,Home,NaN,en,NaN,524073
freq,NaN,47,NaN,226575,NaN,760
mean,142453.983776,NaN,1994.839471,NaN,12.819508,NaN
std,73561.405863,NaN,24.513281,NaN,177.346589,NaN
min,1.000000,NaN,1878.000000,NaN,0.000000,NaN
25%,83783.500000,NaN,1981.000000,NaN,0.000000,NaN
50%,146575.000000,NaN,2004.000000,NaN,0.000000,NaN
75%,205852.500000,NaN,2014.000000,NaN,1.000000,NaN


In [22]:
df_ratings_1.describe(include='all')

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
count,4.000000e+06,4.000000e+06,4000000,47937,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06
unique,NaN,NaN,3822267,47186,NaN,NaN,NaN,NaN
top,NaN,NaN,2020-01-13 19:30:05,8/10,NaN,NaN,NaN,NaN
freq,NaN,NaN,27,39,NaN,NaN,NaN,NaN
mean,2.299874e+06,3.800336e+00,NaN,NaN,5.687500e-04,1.136675e-02,4.960091e+07,5.079680e+03
std,1.329212e+06,1.052033e+00,NaN,NaN,3.469765e-02,1.775135e-01,2.888530e+07,8.526068e+03
min,2.000000e+00,1.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,5.700000e+01,1.000000e+00
25%,1.143354e+06,3.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,2.450126e+07,3.250000e+02
50%,2.296234e+06,4.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,4.922351e+07,1.657000e+03
75%,3.462127e+06,5.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,7.458824e+07,3.434000e+03


In [23]:
df_ratings_2.describe(include='all')

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
count,4.000000e+06,4.000000e+06,4000000,97681,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06
unique,NaN,NaN,3870859,94370,NaN,NaN,NaN,NaN
top,NaN,NaN,2020-01-13 19:30:06,.,NaN,NaN,NaN,NaN
freq,NaN,NaN,55,325,NaN,NaN,NaN,NaN
mean,6.938394e+06,3.592462e+00,NaN,NaN,2.829750e-03,3.098050e-02,4.952727e+07,1.930825e+04
std,1.371393e+06,1.106047e+00,NaN,NaN,8.876792e-02,3.222627e-01,2.891197e+07,2.583543e+04
min,4.588504e+06,1.000000e+00,NaN,NaN,0.000000e+00,-1.000000e+00,4.990000e+02,1.000000e+00
25%,5.752759e+06,3.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,2.450200e+07,1.309000e+03
50%,6.922370e+06,4.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,4.884939e+07,4.159000e+03
75%,8.129423e+06,4.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,7.465211e+07,2.984100e+04


In [1]:
df_ratings_3.describe(include='all')

NameError: name 'df_ratings_3' is not defined

In [16]:
df_ratings_4.describe(include='all')

,rating_id,rating_score,rating_timestamp_utc,critic,critic_comments,critic_likes,user_id,movie_id
count,3.519962e+06,3.519962e+06,3519962,3519962.0,3.519962e+06,3.519962e+06,3.519962e+06,3.519962e+06
unique,NaN,NaN,3430391,223923.0,NaN,NaN,NaN,NaN
top,NaN,NaN,2009-01-06 00:00:00,0.0,NaN,NaN,NaN,NaN
freq,NaN,NaN,267,3249606.0,NaN,NaN,NaN,NaN
mean,1.771008e+07,3.557458e+00,NaN,NaN,7.463433e-03,6.615554e-02,4.995719e+07,7.672412e+04
std,1.319339e+06,1.111050e+00,NaN,NaN,2.103411e-01,4.826350e-01,2.874204e+07,7.701689e+04
min,1.533790e+07,1.000000e+00,NaN,NaN,0.000000e+00,-9.000000e+00,1.760000e+02,1.000000e+00
25%,1.657406e+07,3.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,2.533110e+07,3.373000e+03
50%,1.776064e+07,4.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,4.990784e+07,3.530500e+04
75%,1.885155e+07,4.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,7.459382e+07,1.541460e+05


## 4. Normalizacion de Datos

Como pudimos observar en la descripcion, como en la vista de registros, existen muchas columnas con valores nulos y con valores del tipo NaN, los cuales vamos a proceder a corregir. En la carga inicial de los datos con DB Browser, se selecciono los registros que tengan solamente datos por tanto, los valores nulos han quedado descartados.

In [6]:
df_directors = df_directors.fillna(0)

In [19]:
df_users= df_users.fillna(0)

In [27]:
df_movies = df_movies.fillna(0)

In [28]:
df_movies_onstreaming = df_movies_onstreaming.fillna(0)

In [22]:
df_ratings_1 = df_ratings_1.fillna(0)

In [23]:
df_ratings_2 = df_ratings_2.fillna(0)

In [14]:
df_ratings_3 = df_ratings_3.fillna(0)

In [15]:
df_ratings_4 = df_ratings_4.fillna(0)

Ahora, aplicaremos una normalizacion eliminando espacios innecesario por fuera de los datos y convertir todo a mayuscula. 

In [7]:
df_directors = df_directors.applymap(lambda s:s.upper() if type(s) == str else s)
df_directors[df_directors.columns] = df_directors.apply(lambda x: x.str.strip())

In [20]:
df_directors.head()

,director_id,director_name
0,100,WERNER HERZOG
1,"100, 19053","WERNER HERZOG, ANDRE SINGER"
2,"100, 602, 11891, 783, 832","WERNER HERZOG, ANDRZEJ WAJDA, LUIGI COMENCINI,..."
3,"100, 81649","WERNER HERZOG, DENIS REICHLE"
4,"100006, 57351","SHELLY DUNN FREMONT, VINCENT FREMONT"


In [18]:
df_users = df_users.applymap(lambda s:s.upper() if type(s) == str else s)
df_users[df_users.columns] = df_users.apply(lambda x: x.str.strip() if type(s) == str else s)

,director_id,director_name
0,100,WERNER HERZOG
1,"100, 19053","WERNER HERZOG, ANDRE SINGER"
2,"100, 602, 11891, 783, 832","WERNER HERZOG, ANDRZEJ WAJDA, LUIGI COMENCINI,..."
3,"100, 81649","WERNER HERZOG, DENIS REICHLE"
4,"100006, 57351","SHELLY DUNN FREMONT, VINCENT FREMONT"


In [29]:
df_movies = df_movies.applymap(lambda s:s.upper() if type(s) == str else s)
df_movies[df_movies.columns] = df_movies.apply(lambda x: x.str.strip() if type(x) == str else x)

In [30]:
df_movies_onstreaming = df_movies_onstreaming.applymap(lambda s:s.upper() if type(s) == str else s)
df_movies_onstreaming[df_movies_onstreaming.columns] = df_movies_onstreaming.apply(lambda x: x.str.strip() if type(x) == str else x)

In [25]:
df_ratings_1 = df_ratings_1.applymap(lambda s:s.upper() if type(s) == str else s)
df_ratings_1[df_ratings_1.columns] = df_ratings_1.apply(lambda x: x.str.strip() if type(x) == str else x)

MemoryError: Unable to allocate 244. MiB for an array with shape (8, 4000000) and data type object

In [26]:
df_ratings_2 = df_ratings_2.applymap(lambda s:s.upper() if type(s) == str else s)
df_ratings_2[df_ratings_2.columns] = df_ratings_2.apply(lambda x: x.str.strip() if type(x) == str else x)

MemoryError: Unable to allocate 244. MiB for an array with shape (8, 4000000) and data type object

In [27]:
df_ratings_3 = df_ratings_3.applymap(lambda s:s.upper() if type(s) == str else s)
df_ratings_3[df_ratings_3.columns] = df_ratings_3.apply(lambda x: x.str.strip() if type(x) == str else x)

MemoryError: Unable to allocate 244. MiB for an array with shape (8, 4000000) and data type object

In [ ]:
df_ratings_4 = df_ratings_4.applymap(lambda s:s.upper() if type(s) == str else s)
df_ratings_4[df_ratings_4.columns] = df_ratings_4.apply(lambda x: x.str.strip() if type(x) == str else x)

## 4.1 Union del dataframe "movies" con "movies_onstreaming" 

Para generar una nueva entidad llamada "movies_onstreaming". Se realizaran unas limpiezas basicas para hacer el dataset mucho mas preciso para posibles analisis.

In [31]:
df_mov_onstreamig_f = pd.merge(df_movies, df_movies_onstreaming, on=["movie_title"])
df_mov_onstreamig_f.drop_duplicates(keep=False,inplace=True)
df_mov_onstreamig_f.drop(columns=['id'])
df_mov_onstreamig_f.movie_release_year = df_mov_onstreamig_f.movie_release_year.astype(int)
df_mov_onstreamig_f.movie_popularity = df_mov_onstreamig_f.movie_popularity.astype(int)
df_mov_onstreamig_f.head()

,movie_id,movie_title,movie_release_year,movie_title_language,movie_popularity,director_id,id,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney
0,5,PADRE NUESTRO,2007,EN,7,92,3971,2007,18+,6.9,79%,0,1,0,0
1,9,YOUNG YAKUZA,2007,EN,17,127,11727,2007,0,6.4,30%,0,0,1,0
2,12,THE GENERAL,1926,EN,3124,"46, 47",4469,1926,0,8.1,93%,0,0,1,0
3,13727,THE GENERAL,1998,EN,21,10136,4469,1926,0,8.1,93%,0,0,1,0
4,21826,THE GENERAL,2009,EN,1,71531,4469,1926,0,8.1,93%,0,0,1,0


# 5. Guardar dataframe en JSON en S3

In [16]:
import pandas as pd
import boto3
import io

# code to get the df

destination = "json_files/movies_on_streaming.json"

json_buffer = io.StringIO()

df_mov_onstreamig_f.to_json(json_buffer)

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('movie.data.mubi')

my_bucket.put_object(Key=destination, Body=json_buffer.getvalue())



s3_path = '/some/s3/path'
bucket = 'movie.data.mubi'
local_data = {"id": "123", "name": "XYZ", "transaction": [20.0, 30.0]}

s3 = boto3.client('s3')
resp=s3.get_object(Bucket=bucket, Key=s3_path)
data=resp.get('Body')

json_data = json.loads(data)
json_data.append(local_data)
s3.put_object(Bucket=bucket, Key=s3_path, Body=json.dumps(json_data).encode())

ClientError: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

In [35]:
json = df_mov_onstreamig_f.to_json() 
print(json) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

